# --- Day 4: Ceres Search ---
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```
The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```
Take a look at the little Elf's word search. How many times does XMAS appear?

In [ ]:
# Read input from file
input_file = open("puzzle_input/q4_input.txt")
input = input_file.read()
input_file.close()
input

'.M.S......\n..A..MSMS.\n.M.S.MAA..\n..A.ASMSM.\n.M.S.M....\n..........\nS.S.S.S.S.\n.A.A.A.A..\nM.M.M.M.M.\n..........'

In [12]:

# Use a function that will allow us to see the grid in any direction we wish
# (this is such an overkill function)
def iterateGridFromView(grid, dx, dy):
    if dx == 0 and dy == 0:
        raise RuntimeError()
    
    # Pick an edge to start from
    x = 0
    y = 0
    x_size = len(grid[0])
    y_size = len(grid)
    
    if dx != 0 and dy != 0:
        if dx < 0:
           y = y_size - 1
        if dy > 0:
            x = x_size - 1
    else:
        if dx < 0:
            x = x_size - 1
        if dy < 0:
            y = y_size - 1


    run_count = 0
    # Wrap around values
    start_x = x
    start_y = y
    # Run the algorithm
    while run_count < x_size * y_size:
        # Get our current position
        yield grid[y][x]

        # Increment to the next position
        x += dx
        y += dy

        # Wrap around
        if x < 0 or x >= x_size or y < 0 or y >= y_size:
            # Out of bounds. We must move the x and y to the next start position
            # Determine what the next starting positions should be
            # For example, if dx is 1 and dy is -1 then start_y should increase
            # In the same example, if start_y goes out of bounds then increase
            # start_x instead.

            # left
            if dx > 0 and dy == 0:
                start_y += 1
            # right
            if dx < 0 and dy == 0:
                start_y += 1
            # down
            if dx == 0 and dy < 0:
                start_x += 1
            # up
            if dx == 0 and dy > 0:
                start_x += 1

            # Top left
            if dx > 0 and dy < 0:
                start_y += 1
                if start_y >= y_size:
                    start_y = y_size - 1
                    start_x += 1
            # Top right
            if dx > 0 and dy > 0:
                start_x -= 1
                if start_x < 0:
                    start_x = 0
                    start_y += 1
            # Bottom right
            if dx < 0 and dy > 0:
                start_y -= 1
                if start_y < 0:
                    start_y = 0
                    start_x -= 1
            # Bottom left
            if dx < 0 and dy < 0:
                start_x += 1
                if start_x >= x_size:
                    start_x = x_size - 1
                    start_y -= 1
            
            # Reset x and y positions to new x y 
            x = start_x
            y = start_y

            # Suggest to the caller that we are done with one "line" and must wrap
            yield '\n'
        run_count += 1
    return

In [13]:
import re

# Use 2d array to make this easier
grid = list(map(list, input.splitlines()))

# Try to view it from 8 different angles (pointing right and going clockwise)
views = [(1,0), (1,1), (0,1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

allViews = ''
for dx, dy in views:
    print("dx", dx, "dy", dy)
    print(len(re.findall("XMAS", ''.join(iterateGridFromView(grid, dx, dy)))))
    print()
    allViews += ''.join(iterateGridFromView(grid, dx, dy)) + '\n'

# Now with every single view as one string, we can just do a regex count
len(re.findall("XMAS", allViews))

dx 1 dy 0
225

dx 1 dy 1
366

dx 0 dy 1
222

dx -1 dy 1
399

dx -1 dy 0
205

dx -1 dy -1
446

dx 0 dy -1
220

dx 1 dy -1
434



2517

# --- Part Two ---
The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:

```
M.S
.A.
M.S
```
Irrelevant characters have again been replaced with . in the above diagram. Within the X, each MAS can be written forwards or backwards.

Here's the same example from before, but this time all of the X-MASes have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```
In this example, an X-MAS appears 9 times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an X-MAS appear?

In [22]:
# Use 2d array to make this easier
grid = list(map(list, input.splitlines()))

def isBounded(grid, x, y):
    return not (y < 0 or x < 0 or y >= len(grid) or x >= len(grid[y]))
        

def makesAnX(grid, x, y):
    if not isBounded(grid, x, y):
        return False
    m_count = 0
    s_count = 0
    if grid[y][x] == 'A':
        for dx, dy in [(-1, -1), (1, -1), (-1, 1), (1, 1)]:
            if not isBounded(grid, x + dx, y + dy):
                return False
            if grid[y + dy][x + dx] == 'M':
                m_count += 1
            elif grid[y + dy][x + dx] == 'S':
                s_count += 1
        if grid[y + 1][x + 1] != grid[y - 1][x - 1] and m_count == 2 and s_count == 2:
            return True
    return False


xmas_count = 0

for y, row in enumerate(grid):
    for x, _ in enumerate(row):
        if makesAnX(grid, x, y):
            xmas_count += 1

xmas_count



9